This notebook is to be used with the primitive remote control server implemented in lib/server.h. It is encouraged to rewrite the code's control logic for one's convenience.

In [ ]:
import socket
import numpy as np

# ipaddress = "192.168.1.199"
ipaddress = "127.0.0.1"
# Connect the socket to the port where the server is listening
server_address = (ipaddress, 56456)

empty_list = [[],[],[],[]]

def send_freqs(static_freq,dynamic_freq_init=empty_list,dynamic_freq_final=empty_list,static_amp=empty_list,dynamic_amp=empty_list):
    amp_flag = 0
    string = ""
    if (static_amp == empty_list) and (dynamic_amp == empty_list):
        string += "0,"
    else:
        amp_flag = 1
        string += "1,"
    if dynamic_freq_init == empty_list:
        for i in range(4):
            string += str(len(static_freq[i]))+","
        string += "0,0,0,0,"
        for i in range(4):
            for j in range(len(static_freq[i])):
                try:
                    static_freq[i][j] = int(static_freq[i][j])
                except:
                    raise ValueError("Error: Static frequencies must be numbers!")
                string += str(static_freq[i][j])+","
    else:
        for i in range(4):
            if len(dynamic_freq_init[i])!=len(dynamic_freq_final[i]):
                raise ValueError("Error: Number of initial and final frequencies don't match!")
            string += str(len(static_freq[i])+len(dynamic_freq_init[i]))+","
        for i in range(4):
            string += str(len(dynamic_freq_init[i])) + ","
        for i in range(4):
            for j in range(len(static_freq[i])):
                try:
                    static_freq[i][j] = int(static_freq[i][j])
                except:
                    raise ValueError("Error: Static frequencies must be numbers!")
                string += str(static_freq[i][j])+","
            for k in range(len(dynamic_freq_init[i])):
                try:
                    dynamic_freq_init[i][k] = int(dynamic_freq_init[i][k])
                except:
                    raise ValueError("Error: Dynamic initial frequencies must be numbers!")
                string += str(dynamic_freq_init[i][k]) + ","
        for i in range(4):
            for k in range(len(dynamic_freq_init[i])):
                try:
                    dynamic_freq_final[i][k] = int(dynamic_freq_final[i][k])
                except:
                    raise ValueError("Error: Dynamic final frequencies must be numbers!")
                string += str(dynamic_freq_final[i][k]) + ","
        counter = 0
        for i in range(4):
            counter += len(static_freq[i])
            for k in range(len(dynamic_freq_init[i])):
                string += str(counter) + ","
                counter += 1
    if amp_flag:
        amp_lists = []
        for i in range(4):
            if len(static_amp[i]) != len(static_freq[i]):
                raise ValueError("Error: Number of static amplitudes does not match number of frequencies!")
            if len(dynamic_amp[i]) != len(dynamic_freq_final[i]):
                raise ValueError("Error: Number of dynamic amplitudes does not match number of frequencies!")
            amp_list=[]
            amp_list += static_amp[i]
            amp_list += dynamic_amp[i]
            try:
                amp_list = np.array(amp_list,dtype=np.float64)/np.sum(amp_list)*len(amp_list)
            except:
                raise ValueError("Error: Amplitudes must be a array of numbers!")
            amp_lists.extend(amp_list)
        for iter in range(len(amp_lists)):
            string += "{:.3f}".format(amp_lists[iter]) + ","
    string += "go"
    clientsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    clientsock.connect(server_address)
    clientsock.send(string.encode())
    clientsock.close()


def send_update(selected_num, dynamic_num, selected_indexes, dest_freqs, dynamic_indexes):
    sendstr = ""
    for i in range(4):
        sendstr += str(selected_num[i]) + ","
    for i in range(4):
        sendstr += str(dynamic_num[i]) + ","
    for i in range(len(selected_indexes)):
        sendstr += str(selected_indexes[i]) + ","
    for i in range(len(dest_freqs)):
        sendstr += str(dest_freqs[i]) + ","
    for i in range(len(dynamic_indexes)):
        sendstr += str(dynamic_indexes[i]) + ","
    sendstr += "up"
    clientsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    clientsock.connect(server_address)
    clientsock.send(sendstr.encode())
    clientsock.close()

def send_stop():
    clientsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    clientsock.connect(server_address)
    clientsock.send("stop".encode())
    clientsock.close()    

def send_amp(amp):
    clientsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    clientsock.connect(server_address)
    str = str(int(amp))
    clientsock.send(str+",amp_set".encode())
    clientsock.close()

def send_continue():
    clientsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    clientsock.connect(server_address)
    clientsock.send("continue".encode())
    clientsock.close()    

In [ ]:
static_freqs = [[], # Frequencies of the static tones
                [],
                [99830000,100e6],
                [80e6,85e6,95e6,105e6]
                ]

# dynamic_freqs_init = [[], # Initial frequencies of the dynamic tones
#                       [],
#                       [],
#                       [70e6]]

# dynamic_freq_final = [[], # Final frequencies of the dynamic tones
#                       [],
#                       [],
#                       [110e6]]

static_amp = [[], # Amplitudes of the static tones
              [],
              [1,1.5],
            # test_amps_x.tolist(),
              [5,1.5,2.5,2],
              # test_amps_y.tolist(),
              ]

# dynamic_amp = [[], # Amplitudes of the dynamic tones
#                [],
#                [],       
#                [2]]



try: # Required argument is static_freq; 
     # Optional arguments inludes dynamic_freq_init, dynamic_freq_final, static_amp, dynamic_amp
    send_freqs(static_freqs) # Create static tones 
    # send_freqs(static_freqs,static_amp=static_amp) # Create static tones with different amplitudes
    # send_freqs(static_freqs,dynamic_freqs_init,dynamic_freq_final,static_amp,dynamic_amp) # Create static and dynamic tones
except Exception as e:
    print(e)


In [ ]:
selected_num = [0,0,1,2] # One tone in channel 2 and two tones in channel 3
dynamic_num = [0,0,0,1] # One dynamic tone
selected_indexes = [2,4,5] # select the 2nd, 4th and 5th tone to keep
dest_freqs = [100e6] # Set the destination frequency of the dynamic tone
dynamic_indexes = [2] # select the 3rd selected tone to chirp
send_update(selected_num, dynamic_num, selected_indexes, dest_freqs, dynamic_indexes)